<a href="https://colab.research.google.com/github/ArjunBabu3m3/Big-Data/blob/master/BigData_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Install Java JDK
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Download Spark
!wget -q http://apache.mirrors.pair.com/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz

# Untar the zip file
!tar -xvf spark-2.4.5-bin-hadoop2.7.tgz

In [0]:
# Importing the MovieLens dataset
!wget http://files.grouplens.org/datasets/movielens/ml-latest.zip

--2020-05-07 22:52:15--  http://files.grouplens.org/datasets/movielens/ml-latest.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277113433 (264M) [application/zip]
Saving to: ‘ml-latest.zip.1’

ml-latest.zip.1     100%[===================>] 264.28M   115MB/s    in 2.3s    

2020-05-07 22:52:18 (115 MB/s) - ‘ml-latest.zip.1’ saved [277113433/277113433]



In [0]:
# Unzip the file
!unzip ml-latest.zip

In [0]:
# Install findspark
!pip install -q findspark

# Setup environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

# Spark Session Initialization
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [0]:
# Mention the Path of the u.data file
pathToFile ='/content/ml-latest/ratings.csv'

# Read the data file


In [0]:
data1 = spark.read.format('com.databricks.spark.csv').\
                               options(header='true', \
                               inferschema='true').\
                load(pathToFile,header=True)

In [0]:
# print the schema 
data1.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [0]:
#Check the first 20 rows of the data
data1.show()

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    307|   3.5|1256677221|
|     1|    481|   3.5|1256677456|
|     1|   1091|   1.5|1256677471|
|     1|   1257|   4.5|1256677460|
|     1|   1449|   4.5|1256677264|
|     1|   1590|   2.5|1256677236|
|     1|   1591|   1.5|1256677475|
|     1|   2134|   4.5|1256677464|
|     1|   2478|   4.0|1256677239|
|     1|   2840|   3.0|1256677500|
|     1|   2986|   2.5|1256677496|
|     1|   3020|   4.0|1256677260|
|     1|   3424|   4.5|1256677444|
|     1|   3698|   3.5|1256677243|
|     1|   3826|   2.0|1256677210|
|     1|   3893|   3.5|1256677486|
|     2|    170|   3.5|1192913581|
|     2|    849|   3.5|1192913537|
|     2|   1186|   3.5|1192913611|
|     2|   1235|   3.0|1192913585|
+------+-------+------+----------+
only showing top 20 rows



In [0]:
# We do not require timestamp column, so dropping it
data2 = data1.drop("timestamp")

#print the schema now and check that timestamp column is dropped
data2.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)



In [0]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [0]:
# Randomly split the data into train and test where 80% data is in train and remaining is test
train, test = data2.randomSplit([0.8, 0.2])

In [0]:
# Build a recommendation model using Alternating Least Squares
#als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", nonnegative=True)
#model = als.fit(train)

In [0]:
# Build a recommendation model using Alternating Least Squares
# Evaluate the model by computing the RMSE on the test data
model = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", nonnegative=True, coldStartStrategy="drop").fit(train)
from pyspark.ml.evaluation import RegressionEvaluator
evaluator=RegressionEvaluator(metricName="rmse",labelCol="rating",predictionCol="prediction")
predictions=model.transform(test)
rmse=evaluator.evaluate(predictions)

# print the RMSE
print("New RMSE: ", evaluator.evaluate(model.transform(test)))

New RMSE:  0.8226236096293528


Trying to save the model using pickle(Dint work)

In [0]:
#import pickle

In [0]:
#filename = 'model.sav'
#pickle.dump(model, open(filename, 'wb'))

In [0]:
#loaded_model = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(X_test, Y_test)
#print(result)

In [0]:
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.tuning import ParamGridBuilder

# Now we try to improve the performance of the original model using cross validation and solve the cold-start problem.
# we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics

model = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", nonnegative = True, coldStartStrategy="drop")

# We use a ParamGridBuilder to construct a grid of parameters to search over.
paramGrid = ParamGridBuilder() \
    .addGrid(model.regParam, [0.1, 0.01, 0.001]) \
    .addGrid(model.rank, [5, 10, 15]) \
    .build()

# Define a cross-validator object
# A CrossValidator requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
crossvalidation = CrossValidator(estimator = model,
                     estimatorParamMaps = paramGrid,
                     evaluator = evaluator,
                     numFolds=5)

In [0]:
# Run cross-validation, and choose the best set of parameters.
Best_model = crossvalidation.fit(train).bestModel

In [0]:
# Calculate the RMSE on test data using the best set of parameters obtained after cross validation
print("Best RMSE value is: ", evaluator.evaluate(Best_model.transform(test)))

Best RMSE value is:  0.8231774775983364


In [0]:
# Now there is an improvement in the performance and we can see that RMSE value is 0.9218

In [0]:
# Get top 10 movie recommendations for all users
movie_recommendation = Best_model.recommendForAllUsers(10)
movie_recommendation.show()
import pandas as pd
movie_recommendation = movie_recommendation.toPandas()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|   148|[[185659, 5.65381...|
|   463|[[151989, 3.72915...|
|   471|[[177209, 5.15297...|
|   496|[[185659, 5.62696...|
|   833|[[151989, 4.75969...|
|  1088|[[185659, 5.40584...|
|  1238|[[185659, 3.78415...|
|  1342|[[126737, 3.06373...|
|  1580|[[185659, 6.23385...|
|  1591|[[185659, 6.31438...|
|  1645|[[185659, 6.54202...|
|  1829|[[185659, 7.08875...|
|  1959|[[151989, 6.04134...|
|  2122|[[185659, 5.71250...|
|  2142|[[188923, 6.47613...|
|  2366|[[185659, 6.90093...|
|  2659|[[151989, 4.99159...|
|  2866|[[177209, 6.79304...|
|  3175|[[177209, 2.82178...|
|  3749|[[183947, 6.16469...|
+------+--------------------+
only showing top 20 rows



In [0]:
users = []
recommendations = []

# For the entire data iterate 
for i in range(len(movie_recommendation)):

  # First we add user_id to the list
  users.append(movie_recommendation.iloc[i,0])
  user_recs = "" 

  # Fetching only the item ID's from the recommendations
  for item in movie_recommendation.iloc[i,1]:
     user_recs = user_recs + ", " + str(item.asDict()["movieId"])

  # Append the itemID's to a list
  recommendations.append(user_recs[2:])

#Convert to dataframe
recommendations_df = pd.DataFrame(data = zip(users, recommendations), columns=["user", "MovieID"])

In [0]:
#Check the users and the top 10 movie recommendations for the first 10 users
recommendations_df.head(10)

,user,MovieID
0,148,"185659, 151989, 3226, 177209, 155923, 187873, ..."
1,463,"151989, 185659, 101862, 142608, 166267, 135057..."
2,471,"177209, 151989, 185659, 190707, 159761, 144202..."
3,496,"185659, 151989, 177209, 66389, 183947, 173737,..."
4,833,"151989, 178147, 121919, 101706, 190707, 184299..."
5,1088,"185659, 94339, 177209, 152711, 178147, 121919,..."
6,1238,"185659, 151989, 66811, 125279, 135735, 193253,..."
7,1342,"126737, 149570, 8945, 163781, 84996, 99724, 91..."
8,1580,"185659, 177209, 183947, 151989, 159761, 157791..."
9,1591,"185659, 177209, 151989, 117352, 178147, 121919..."


In [0]:
#write to a text file
recommendations_df.to_csv('/content/recommendation_project.txt', sep='\t', index=False)